# Hessen

Every federal state is represented by its own input directory and is processed into a NUTS level 2 directory containing a sub-folder for each discharge location. These folder names are derived from NUTS and reflect the CAMELS id. The NUTS level 2 code for Hessen is `DE7`.

To pre-process the data, you need to write (at least) two functions. One should extract all metadata and condense it into a single `pandas.DataFrame`. This is used to build the folder structure and derive the ids.
The second function has to take an id, as provided by the state authorities, called `provider_id` and return a `pandas.DataFrame` with the transformed data. The dataframe needs the three columns `['date', 'q' | 'w', 'flag']`.

For easier and unified output handling, the `camelsp` package contains a context object called `Bundesland`. It takes a number of names and abbreviations to identify the correct federal state and returns an object that holds helper and save functions.

The context saves files as needed and can easily be changed to save files with different strategies, ie. fill missing data with NaN, merge data into a single file, create files for each variable or pack everything together into a netcdf.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pandas.errors import ParserError
import os
from pprint import pprint
from tqdm import tqdm
from typing import Union, Dict
import zipfile
from datetime import datetime as dt
from io import StringIO
import warnings

from camelsp import Bundesland, Station

The context can also be instantiated as any regular Python class, ie. to load only the default input data path, that we will user later.

In [2]:
# the context also makes the input path available, if camelsp was install locally
BASE = Bundesland('Hessen').input_path
BASE

'/home/camel/camelsp/input_data/HE_Hessen'

### Metadata reader

Define the function that extracts / reads and eventually merges all metadata for this federal state. You can develop the function here, without using the Bundesland context and then later use the context to pass extracted metadata. The Context has a function for saving *raw* metadata, that takes a `pandas.DataFrame` and needs you to identify the id column.
Here, *raw* refers to provider metadata, that has not yet been transformed into the CAMELS-de Metadata schema.

In [3]:
# define the function 
def read_meta(base_path) -> pd.DataFrame:
    path = os.path.join(base_path,'raw_data', 'Pegel_für_Camels.xlsx')
    meta = pd.read_excel(path)
    return meta

# test it here
metadata = read_meta(BASE)

metadata

,Pegelname,Gewässer,Messstellen Nr.,Q seit AJ,Koordinaten X,Koordinaten Y,Koordinaten-system,Höhe [m ü. NN],Größe des Einzugsge-biets [km²]
0,Adelshausen,Pfieffe,42780500,1981,539203.32,5662292.03,UTM_ETRS89,171.238,116.16
1,Alsfeld,Schwalm,42880458,1968,520113.63,5622930.81,UTM_ETRS89,237.682,131.41
2,Angenrod,Antreff,42881009,1977,514566.84,5623429.65,UTM_ETRS89,280.930,58.74
3,Aßlar,Dill,25842500,1963,462040.59,5603547.93,UTM_ETRS89,153.030,693.63
4,Auhammer,Eder,42810204,1960,473648.32,5653835.87,UTM_ETRS89,298.216,488.94
...,...,...,...,...,...,...,...,...,...
92,Unter-Schmitten,Nidda,24810600,1967,501734.70,5587094.18,UTM_ETRS89,132.300,124.00
93,Uttershausen,Schwalm,42882806,1958,522997.73,5657842.91,UTM_ETRS89,164.411,984.98
94,Weilers,Bracht,24782800,1972,522136.45,5569511.02,UTM_ETRS89,140.790,111.90
95,Windecken,Nidder,24861407,1956,491018.79,5563393.68,UTM_ETRS89,112.620,392.60


In [4]:
# the id column will be Messstellen Nr.
id_column = 'Messstellen Nr.'

## file extract and parse

Should be straightforward, really nice data

In [5]:
def extract_file(nr: Union[int, str], variable: str, input_path: str) -> pd.DataFrame:

    # build the path to the correct subfolder:
    path = os.path.join(input_path, variable, f"{nr}_{variable.upper()}.txt")

    # check file
    if not os.path.exists(path):
        return pd.DataFrame(columns=['date', variable.lower(), 'flag'])
    
    # read
    raw = pd.read_csv(path, skiprows=4, encoding='latin1', sep=' ', header=None, na_values=-777)

    df = pd.DataFrame({
        'date': [dt.strptime(str(_)[:8], '%Y%m%d') for _ in raw.iloc[:, 0]],
        variable.lower(): raw.iloc[:, 1],
    })

    # set flag True before 2018 (see Anfrage-Camels-HUIG-79c1801_Spi.pdf)
    df['flag'] = df['date'] < pd.to_datetime('2018-01-01')

    return df

extract_file(41980355, 'q', BASE)

,date,q,flag
0,1957-11-01,0.106,True
1,1957-11-02,0.092,True
2,1957-11-03,0.092,True
3,1957-11-04,0.078,True
4,1957-11-05,0.078,True
...,...,...,...
23432,2021-12-27,0.084,False
23433,2021-12-28,0.097,False
23434,2021-12-29,0.146,False
23435,2021-12-30,0.210,False


### Finally run

Now, the Q and W data can be extracted along with the metadata. The cool thing is, that all the id creation, data creation, merging and the mapping from our ids to the original ids and files is done by the context. This is helpful, as we less likely screw something up.

In [6]:
with Bundesland('Hessen') as bl:
    # save the metadata
    bl.save_raw_metadata(metadata, id_column, overwrite=True)

    # for reference, call the nuts-mapping as table
    nuts_map = bl.nuts_table
    print(nuts_map.head())

    
    with warnings.catch_warnings(record=True) as warns:
        for provider_id in tqdm(metadata[id_column].values.astype(str)):
            # get q
            q = extract_file(provider_id, 'q', bl.input_path)
            w = extract_file(provider_id, 'w', bl.input_path)

            bl.save_timeseries(q, provider_id)
            bl.save_timeseries(w, provider_id)

        # check if there were warnings (there are warnings)
        if len(warns) > 0:
            log_path = bl.save_warnings(warns)
            print(f"There were warnings during the processing. The log can be found at: {log_path}")


    nuts_id provider_id                              path
0  DE710000    42780500  ./DE7/DE710000/DE710000_data.csv
1  DE710010    42880458  ./DE7/DE710010/DE710010_data.csv
2  DE710020    42881009  ./DE7/DE710020/DE710020_data.csv
3  DE710030    25842500  ./DE7/DE710030/DE710030_data.csv
4  DE710040    42810204  ./DE7/DE710040/DE710040_data.csv


  0%|          | 0/97 [00:00<?, ?it/s]

100%|██████████| 97/97 [01:07<00:00,  1.44it/s]
